In [3]:
numbers = '123456789'
chars = 'ABCDEFGHI'
import itertools
class Sudoku:
    def __init__(self, board):
        self.board = board 
        self.variables, self.domains, self.constraints, self.neighbors, self.pruned  = list(), {}, list(),{}, {}
        self.prepare(board)
    def prepare(self, board):
        
        # initialize all the the attrbiutes in the class 
        game = list(board)
        self.variables = self.combine(chars, numbers)
        self.domains = {v: list(range(1,10)) if game[i] == '0' else [int(game[i])] for i, v in enumerate(self.variables)}
        self.pruned = {v: [] if game[i]==0 else [int(game[i])] for i, v in enumerate(self.variables)}
        self.build_constraints()
        self.build_neighbors()
        
    def build_constraints(self):
        # make blocks of the rows and columns and the blocks of 3x3 grid 
        blocks = ([self.combine(chars, number) for number in numbers]+
                 [self.combine(char, numbers) for char in chars]+
                 [self.combine(char, number) for char in ('ABC','DEF', 'GHI') for number in ('123','456','789')])
        for block in blocks:
            combinations = self.permute(block)
            for combination in combinations:
                if [combination[0], combination[1]] not in self.constraints:
                    self.constraints.append([combination[0], combination[1]])
                                    
        # 
    def build_neighbors(self):
        for x in self.variables:
            self.neighbors[x] = list()
            for c in self.constraints:
                if x == c[0]:
                    self.neighbors[x].append(c[1])
    
                                             
                                
    def solved(self):
        for v in self.variables:
            if len(self.variables[v]) > 1:
                return False 
        return True 
                                             
    
    def complete(self, assignment):
        for x in self.variables:
            if len(self.domains[x]) > 1 and x not in assignment:
                return False 
        
        return True
    
    def consistent(self, assignment, var, value):
        consistent = True
        
        for key, val in assignment.iteritems():
            if val == value and key in self.neighbors[var]:
                consistent = False
        
        return consistent
            
    def assign(self, var, value, assignment):
        assignment[var] = value
        
        self.forward_check(var, value, assignment)
    
    def unassign(self, var, assignment):
        if var in assignment:
            
            for (D,V) in self.pruned[var]:
                self.domains[D].append(V)
            
            self.pruned[var] = []
            
            del assignment[var]
                
    
    def forward_check(self, var, value, assignment):
        for neighbor in self.neighbors[var]:
            if neighbor not in assignment:
                if value in self.domains[neighbor]:
                    self.domains[neighbor].remove(value)
                    self.pruned[var].append((neighbor, value))
                          
    
    def constraint(self, xi, xj):
        return xi != xj
    
    def combine(self, var1, var2):
        return [a + b for a in var1 for b in var2]
        
    def permute(self, iterable):
        result = []
        for i in range(len(iterable)+1):
            if i==2:
                for subset in itertools.permutations(iterable,i):
                    result.append(subset)
        return result
    
    def conflicts(self, sudoku, var, value):
        count = 0
        
        for n in sudoku.neighbors[var]:
            if len(sudoku.domains[n]) > 1 and val in sudoku.domains[n]:
                count += 1
        
        return count
    def out(self, mode):
        if mode == 'console':
            
            for var in self.variables:
                sys.stdout.write(str(self.domains[var][0]))
        elif mode == 'file':
            return 

In [32]:
def ARC_3(sudoku):
    queue = list(sudoku.constraints)
#     print(queue)
    while queue:
        xi,xj = queue.pop(0)
        
        if revise(sudoku,xi,xj):
            if len(sudoku.domains[xi]) == 0:
                return False
            for xk in sudoku.neighbors[xi]:
                if xk != xi:
                    queue.append([xk,xj])

    return True
def revise(sudoku, xi,xj):
    revised = True 
    for x in sudoku.domains[xi]:
        if not any([sudoku.constraint(x,y) for y in sudoku.domains[xj]]):
            sudoku.domains[xi].remove(x)
            revised = True 
    return revised
def backtrack(assignment, sudoku):

    if len(assignment) == len(sudoku.variables):
        return assignment

    var = select_unassigned_variable(assignment, sudoku)

    for value in order_domain_values(sudoku, var):

        if sudoku.consistent(assignment, var, value):

            sudoku.assign(var, value, assignment)

            result = backtrack(assignment, sudoku)
            if result:
                return result

            sudoku.unassign(var, assignment)

    return False
# def backtrack(assignment, sudoku):
#     if len(assignment) == len(sudoku.variables):
#         return assignment
    
#     var = select_unassigned_variable(assignment, sudoku)
    
#     for value in order_domain_values(sudoku, var):
#         if sudoku.consistent(assignment, var, value):
#             sudoku.assign(var, value, assignment)
#             result = backtrack(assignment, sudoku)
            
#             if result:
#                 return result 
#             sudoku.unassign(var, assignment)
    
#     return False
    
    
def select_unassigned_variable(assignment, sudoku):
    unassigned = [v for v in sudoku.variables if v not in assignment]
    return min(unassigned, key=lambda var: len(sudoku.domains[var]))


# Least Constraining Value heuristic
# Prefers the value that rules out the fewest choices for the neighboring variables in the constraint graph.
def order_domain_values(sudoku, var):
    if len(sudoku.domains[var]) == 1:
        return sudoku.domains[var]

    return sorted(sudoku.domains[var], key=lambda val: sudoku.conflicts(sudoku, var, val))



In [33]:
# data = 'https://raw.githubusercontent.com/dpalmasan/artificial_intelligence_columbia_edx/master/Project4/sudokus_start.txt'
def main():
    with open('sudokus_start.txt') as data:
        for board in data:
            sudoku = Sudoku(board)
            if ARC_3(sudoku):
                if sudoku.solved():
                    print('heey')
                    output = open('output', 'w')
                    for var in sudoku.variables:
                        output.write(str(sudoku.domains[var][0]))
                    output.close()
            else:
                    assignment = {}
                    for x in sudoku.variables:
                        if len(sudoku.domains[x]) == 1:
                            assignment[x] = sudoku.domains[x][0]
                
                    assignment = backtrack(assignment, sudoku)
                    print(assignment)
                    if assignment:
#                         print(sudoku.domains)
                        for d in sudoku.domains:
                            
                            sudoku.domains[d] = assignment[d] if len(d) > 1 else sudoku.domains[d]
                        else:
                            print("No solution exists")
    
#                 for d in sudoku.domains:
#                     sudoku.domains[d] = assignment[d] if len(d) > 1 else sudoku.domains[d]
                
                    if assignment:
                        
        
                        output = open('finish.txt', 'w')
                        for var in sudoku.variables:
                            output.write(str(sudoku.domains[var]))
                        output.close()

                    else:
                        print("No solution exists")
                    
main()    

False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists


False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists
False
No solution exists


In [24]:
d = open('dil.txt', 'w')
d.write('yooo')
d.close()

SyntaxError: invalid syntax (<ipython-input-22-f2228f2974eb>, line 1)